# Supplementary figure for the robustness to Gaussian noise

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os, json
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full = np.asarray(json.load(f))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

In [ ]:
# Extra color here
noise_clr = "xkcd:sage"
noise_clr = back_palette[0]
noise_clr = mpl.colors.to_hex("xkcd:ultramarine")
noise_clr = mpl.colors.to_hex("xkcd:dark sea green")
print(noise_clr)

In [ ]:
n_neu = np.load(os.path.join(data_folder, "noise",
                    "sample_turbulent_gaussnoise_ibcm_simulation.npz"))["hbars_gammaser"].shape[1]
print(n_neu)
n_components, n_orn = np.load(os.path.join(data_folder, "noise",
                    "sample_turbulent_gaussnoise_ibcm_simulation.npz"))["back_vecs"].shape

In [ ]:
# Extra aesthetic parameters for this figure

# More legend rcParams: make everything smaller by 30 %
plt.rcParams["patch.linewidth"] = 0.75
legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
for k in legend_rc:
    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]

new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
neuron_styles = linestyles + [(0, (1, 2, 1, 2))]

In [ ]:
def l2_norm(vecs, axis=-1):
    r""" Computes l2 norm of vectors stored along the last axis of vecs.
    Args:
        vecs can be either a single vector (1d) or an  arbitrary array of vectors,
            where the last dimension indexes elements of vectors.
        axis (int): which axis to sum along.

    Returns: array of distances of same shape as vecs
        except for the summation axis, removed.
    """
    return np.sqrt(np.sum(vecs**2, axis=axis))

## Panel A: bits to illustrate the white noise added to OSNs

In [ ]:
turbulent_sample = np.load(pj(data_folder, "sample_turbulent_background.npz"))
conc_ser = turbulent_sample["nuser"][:, :, 1]
tser = np.arange(conc_ser.shape[0])
conc_ser.shape

In [ ]:
tslice = slice(65200, 66200, 1)
fig, axes = plt.subplots(3, 1, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.25, plt.rcParams["figure.figsize"][1]*0.2*3)
lwidth = 0.5
for i in range(3):
    axes.flat[i].plot(tser[tslice], conc_ser[tslice, 2-i], lw=lwidth, color=back_palette[1], alpha=1.0)
    axes.flat[i].axis("off")
fig.tight_layout(h_pad=0.0)
if do_save_plots:
    fig.savefig(pj(panels_folder, "resources", "odors_turbulent_concentrations.pdf"), transparent=True)
plt.show()
plt.close()

In [ ]:
tslice = slice(65200, 66200, 1)
rng = np.random.default_rng(0x1dcc5dc76431cb33656af4e10a464d90)
samplesize = 30
for i in range(4):
    fig, ax = plt.subplots()
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.2, plt.rcParams["figure.figsize"][1]*0.2)
    ax.plot(tser[:samplesize], rng.standard_normal(size=samplesize), lw=lwidth, color=noise_clr)
    ax.axis("off")
    fig.tight_layout()
    if do_save_plots:
        fig.savefig(pj(panels_folder, "resources", f"osn_gaussian_noise_{i}.pdf"), transparent=True)
    plt.show()
    plt.close()

## Panel B: OSN time series and concentration statistics

In [ ]:
# Load example background time series
ex = np.load(os.path.join(data_folder, "noise", "sample_turbulent_gaussnoise_ibcm_simulation.npz"))
print(list(ex.keys()))

# Load the sample background time series without skips
tser_noskp = ex["tser_noskp"]  # Unskip this
bkvecser_noskp = ex["bkvecser_noskp"]
bkvecser_odors_noskp = ex["bkvecser_odors_noskp"]

# Time units for plotting, in ms
dt_u = 10.0  # ms

In [ ]:
# Plot time series of four OSNs to match the cartoon in panel A
fig, axes = plt.subplots(4, 1, sharex=True, sharey="col")
#fig.set_size_inches(plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])

# Plot time series
t_window = slice(0, 1500)
tser_ex = tser_noskp[t_window]
choice_osns = list(range(1, 5))
osns_ex = bkvecser_noskp[t_window, choice_osns]
odors_ex = bkvecser_odors_noskp[t_window, choice_osns]
for i in range(4):
    ax = axes.flat[i]
    lbl1 = "Turbulent" if i == 0 else None
    lbl2 = "Plus noise" if i == 1 else None
    ax.plot(tser_ex * dt_u / 1000.0, osns_ex[:, i], color=noise_clr, lw=0.75, label=lbl2)
    ax.plot(tser_ex * dt_u / 1000.0, odors_ex[:, i], color=back_palette[1], lw=1.0, label=lbl1)
    ax.set_ylabel("OSN {}".format(choice_osns[i]), labelpad=0.5)
axes.flat[-1].set_xlabel("Time (s)")

fig.tight_layout()
legends = []
for i in range(2):
    leg = axes.flat[i].legend(frameon=False, fontsize=6, 
                    loc="upper left", bbox_to_anchor=(0.625, 1.4))
    legends.append(leg)

if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_robustness_background_series.pdf"), 
                transparent=True, bbox_inches="tight", bbox_extra_artists=legends)
plt.show()
plt.close()

## Plan panels to make
- C: response of BioPCA and IBCM to parallel and orthogonal noise components
- D: IBCM hgammas
- E: BioPCA eigenvalues
- F: BioPCA alignment
- G: Habituation in one example, all models
- H: Jaccards vs noise amplitude $\sigma_{\mathrm{OSN}}$. Note: on that plot, show the average standard deviation/magnitude of OSN activity in the absence of noise. I assume the performance drops when the noise gets on the same order. Since we are adding noise to each OSN, this is like $\sigma$ times a vector of ones: larger than a unit-normalized odor vector times some concentration $c$. Maybe we should plot the effective norm of the noise, $\sqrt{N} \sigma$? Sum of iid Gaussian.

Plot the average dot product of each IBCM neuron's $\mathbf{m}$ with directions orthogonal to the background manifold as a function of noise, to show the neurons remain specific to actual odors, not noise directions. 


### Recall all that was saved

```
if do_save_outputs:
    fold = os.path.join("..", "results", "for_plots", "noise")
    results_ibcm_filename = "sample_turbulent_gaussnoise_ibcm_simulation.npz"
    save_skp = 10
    np.savez_compressed(
        os.path.join(fold, results_ibcm_filename), 
        tser_noskp=tser_example, 
        bkvecser_noskp=bkvecser_example, 
        bkvecser_odors_noskp=bkvecser_odors_example,
        tser=tser_ibcm[::save_skp], 
        bkvecser=bkvecser_ibcm[::save_skp],
        hbars_gammaser=cbars_gamma[::save_skp],
        back_vecs=back_components, 
        moments_conc=moments_conc,
        hs_hn=hs_hn,
        specif_gammas=specif_gammas, 
        skp=skp*save_skp,
        hser_orthog_back=hser_to_orthog_noise,
        hser_parallel_back=hser_to_parallel_noise,
        orthog_noise_norm=orthog_noise_norm,
        parallel_noise_norm=parallel_noise_norm
    )

    # y series norms
    np.savez_compressed(
        os.path.join(fold, "yser_norm_turbulent_gaussnoise_model_comparison.npz"), 
        **{a:ynorm_series[a][::save_skp] for a in ynorm_series} 
    )

    results_biopca_filename = "sample_turbulent_gaussnoise_biopca_simulation.npz"
    np.savez_compressed(
        os.path.join(fold, results_biopca_filename),
        true_pca_values=true_pca[0],
        true_pca_vectors=true_pca[1], 
        learnt_pca_values=learnt_pca[0][::save_skp],
        learnt_pca_vectors=learnt_pca[1][::save_skp],
        off_diag_l_pca=off_diag_l_avg_abs[::save_skp],
        align_error_pca=align_error_ser[::save_skp]
        hser_orthog_back=hser_to_orthog_noise,
        hser_parallel_back=hser_to_parallel_noise,
        orthog_noise_norm=orthog_noise_norm,
        parallel_noise_norm=parallel_noise_norm
    )
```

## Panel C: $h_\gamma$ series

In [ ]:
ex_ysers = np.load(pj(data_folder, "noise", "yser_norm_turbulent_gaussnoise_model_comparison.npz"))

In [ ]:
back_vecs_example = ex["back_vecs"]
n_b = back_vecs_example.shape[0]
yser_ibcm_example = ex_ysers["ibcm"]
hgammaser_example = ex["hbars_gammaser"]
tser_example = ex["tser"]
n_i_ibcm = hgammaser_example.shape[1]

# Analytical predictions
analytical_hs_hn = ex["hs_hn"]
specif_gammas = ex["specif_gammas"]

# Matrix of all dot products for each neuron with each background odor
analytical_hgammas = np.ones([n_i_ibcm, n_b]) * analytical_hs_hn[1]
analytical_hgammas[np.arange(n_i_ibcm), specif_gammas] = analytical_hs_hn[0]
print(specif_gammas)

In [ ]:
# Show three neurons
fig = plt.figure()
gs = fig.add_gridspec(3, 4)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.25, 
                    plt.rcParams["figure.figsize"][1])
ax = fig.add_subplot(gs[:, :3])
axi = fig.add_subplot(gs[1, 3:])

#ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
t_axis = tser_example*dt_u/1000/60
legend_styles = [[0,]*6, [0,]*6, [0,]*6]
i_highlights = [2, 10, 20]  # Neurons to highlight
neuron_colors3 = neuron_colors_full[[8, 17, 23]]
clr_back = back_palette[-1]
plot_skp = 4
tslice = slice(0, t_axis.shape[0], plot_skp)

# plot all other neurons first, skip some points
for i in range(n_i_ibcm):
    if i in i_highlights: 
        continue
    else: 
        for j in range(n_b):
            ax.plot(t_axis[tslice], hgammaser_example[tslice, i, j], color=clr_back, 
                ls="-", alpha=1.0-0.1*j, lw=plt.rcParams["lines.linewidth"]-j*0.1)

# Now plot the highlighted neuron
for j in range(n_b):
    for i in range(len(i_highlights)):
        li, = ax.plot(t_axis[tslice], hgammaser_example[tslice, i_highlights[i], j], 
                      color=neuron_colors3[i], ls="-", alpha=1.0-0.1*j, 
                      lw=plt.rcParams["lines.linewidth"]-j*0.1)
        legend_styles[i][j] = li

# Annotate with analytical results
for j in range(2):
    ax.axhline(analytical_hs_hn[j], lw=1.0, ls="-.", color="k")
ax.annotate(r"Analytical $h_{\mathrm{ns}}$",  
            xy=(t_axis[-5], analytical_hs_hn[1]-1.6),
            ha="right", va="top", color="k", size=6)
ax.annotate(r"",  
            xy=(t_axis[-1]+1, analytical_hs_hn[1]-0.3), 
            xytext=(t_axis[-5], analytical_hs_hn[1]-2.0),
            ha="right", va="top", color="k", size=6, 
            arrowprops={"color":"k", "headwidth":2.5, "headlength":2.0, "width":0.3})
ax.annotate(r"Analytical $h_\mathrm{sp}$", xy=(t_axis[-5], analytical_hs_hn[0]+0.4), 
            ha="right", va="bottom", color="k", size=6)
ax.set(xlabel="Time (min)", 
       ylabel=r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")
#ax.axhline(analytical_h_saddle, lw=1.0, ls=":", color="grey")
#ax.annotate(r"$h_\mathrm{saddle}$", xy=(t_axis[-150], analytical_h_saddle+0.1), 
#            ha="right", va="bottom", color="grey", size=6)

axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
for i in range(len(i_highlights)):
    for j in range(n_b):
        li = legend_styles[i][j]
        axi.plot([0.0+j, 0.5+j], [0.8*i, 0.8*i], color=li.get_color(), 
                 ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
for side in ["bottom", "left", "top", "right"]:
    axi.spines[side].set_visible(False)
axi.tick_params(axis="both", length=0, pad=3)
axi.set_xticks(np.arange(0.25, n_b+0.25, 2.0))
#axi.set_xticklabels([r"$\mathbf{\bar{m}}^i \cdot \mathbf{s}_a$", r"$\mathbf{\bar{m}} \cdot \mathbf{s}_b$"])
axi.set_xticklabels(list(range(0, n_b, 2)))
axi.set_xlabel(r"Odor $\gamma$", size=6, labelpad=3)
axi.xaxis.set_label_position('top') 
axi.set_yticks([0, 0.8, 1.6])
axi.invert_yaxis()
axi.set_yticklabels(["${}$".format(i) for i in i_highlights])
#axi.set_yticklabels(["Neuron 2", "Neuron 1"])
axi.set_ylabel("Neuron $i$", size=6, labelpad=1)

gs.tight_layout(fig, w_pad=-0.2)
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_robustness_ibcm_hgamma_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel C companion: dot products summary
To really show the specificity. We may not have space to show this. 

In [ ]:
transient = int(3*tser_example.size/4)
hgammas_matrix = np.mean(hgammaser_example[transient:], axis=0)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.6, plt.rcParams["figure.figsize"][1])
# Extent: left, right, bottom, top
# Greyscale version
#ax.imshow(hgammas_matrix, cmap="Greys", aspect=0.6, extent=(0.5, n_b+0.5, 0.5, n_i_ibcm))
#ax.set_xticks(list(range(1, n_b+1)))
# Colorful version: add patches manually with fill_between. 
# Color highlighted neurons, leave others grayscale!
normed_matrix = (hgammas_matrix - hgammas_matrix.min()) / (hgammas_matrix.max() - hgammas_matrix.min())
for i in range(n_i_ibcm):
    # Full rainbow version
    #cmap = sns.light_palette(neuron_colors_full[i], as_cmap=True)
    # Version where only highlights are colored
    if i in i_highlights:
        cmap = sns.light_palette(neuron_colors3[i_highlights.index(i)], as_cmap=True)
    else:
        cmap = sns.color_palette("Greys", as_cmap=True)
    for j in range(n_b):
        ax.fill_between([-0.5+j, 0.5+j], -0.5+i, 0.5+i, color=cmap(normed_matrix[i, j]))
        
ax.set_xlim([-0.6, -0.6+n_b])
ax.set_ylim([-0.6, -0.6+n_i_ibcm])
ax.set_xticks(list(range(0, n_b)))
ax.set_yticks(list(range(0, n_i_ibcm, 2)))
for i, lbl in enumerate(ax.get_yticklabels()):
    if int(lbl.get_text()) in i_highlights:
        clr = neuron_colors3[i_highlights.index(int(lbl.get_text()))]
        lbl.set_color(clr)
        ax.yaxis.get_ticklines()[i].set_color(clr)
ax.set(xlabel=r"Component $\gamma$", ylabel="IBCM neuron index $i$")
cbar = fig.colorbar(mpl.cm.ScalarMappable(
    norm=mpl.colors.Normalize(hgammas_matrix.min(), hgammas_matrix.max()), 
    cmap="Greys"), ax=ax, label=r"Alignments $\bar{h}_{i\gamma}$ after 45 min", aspect=30, pad=0.1)
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_robustness_ibcm_hgamma_matrix.pdf"), 
            transparent=True, bbox_inches="tight", bbox_extra_artists=(cbar.ax,))
plt.show()
plt.close()

## Panel D: BioPCA learning of this background (L, alignment, off-diagonal)
We may not have space to show the alignment error and the off-diagonal elements. 

In [ ]:
ex_pca = np.load(pj(data_folder, "noise", "sample_turbulent_gaussnoise_biopca_simulation.npz"))
true_pca_values = ex_pca["true_pca_values"]
true_pca_vectors = ex_pca["true_pca_vectors"]
learnt_pca_values = ex_pca["learnt_pca_values"]
learnt_pca_vectors = ex_pca["learnt_pca_vectors"]
align_error_ser = ex_pca["align_error_pca"]
yser_pca = ex_ysers["biopca"]
off_diag_l_pca = ex_pca["off_diag_l_pca"]

n_comp = learnt_pca_values.shape[1]

In [ ]:
#pca_palette = sns.cubehelix_palette(start=0.1, rot=0.5, n_colors=n_comp)[::-1]
pca_palette = np.asarray(sns.husl_palette(n_colors=n_comp, h=0.01, s=0.9, l=0.4, as_cmap=False))
#pca_palette = np.concat([pca_palette[::2], pca_palette[1::2]], axis=0)
sns.palplot(pca_palette)

In [ ]:
# First plot: eigenvalues
fig = plt.figure()
gs = fig.add_gridspec(3, 4)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.25, 
                    plt.rcParams["figure.figsize"][1])
ax = fig.add_subplot(gs[:, :3])
axi = fig.add_subplot(gs[:, 3:])

skp_plot = 4
tslice = slice(0, tser_example.shape[0], skp_plot)

biopca_lines = []
for i in range(n_comp):
    li, = ax.plot(tser_example[tslice]*dt_u/1000/60, learnt_pca_values[tslice, i], label="Value {}".format(i),
                  lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=10-i, color=pca_palette[i])
    biopca_lines.append(li)
    if true_pca_values[i] / true_pca_values.max() > 1e-12:
        ax.axhline(true_pca_values[i], ls="--", color=pca_palette[i], 
                   lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=n_comp-i)
ax.set(ylabel="Principal values (diag$(L^{-1})$)", yscale="log", xlabel="Time (min)")

# Custom legend to indicate analytical vs learned
handles = [mpl.lines.Line2D([0], [0], color="grey", ls="-", label=r"BioPCA", 
                            lw=plt.rcParams["lines.linewidth"]), 
          mpl.lines.Line2D([0], [0], color="grey", ls="--", label="True PCA", 
                          lw=plt.rcParams["lines.linewidth"])]
leg = ax.legend(handles=handles, frameon=False, ncols=2, columnspacing=0.8, borderaxespad=0)
leg.set_zorder(30)
ax.set_ylim([ax.get_ylim()[0]*0.8, ax.get_ylim()[1]])

# Small axis for second legend for LN indices
axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
for j in range(n_comp):
    li = biopca_lines[j]
    axi.plot([0.0, 1.0], [0.8*j, 0.8*j], color=li.get_color(), label=str(j),
                 ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
handles, labels = axi.get_legend_handles_labels()
axi.clear()
leg = axi.legend(handles, labels, title="LN index", title_fontsize=6, frameon=False, loc="upper left", borderaxespad=0.0)
axi.axis("off")


fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_robustness_biopca_eigenvalues.pdf"), 
            transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()

In [ ]:
# Alignment error: we may not have space to show this. 
skp_plot = 4
tslice = slice(0, tser_example.shape[0], skp_plot)

fig, axes = plt.subplots(2, 1, sharex=True)
ax = axes[0]
ax.plot(tser_example[tslice]*dt_u/1000/60, align_error_ser[tslice], color="k")
ax.set(yscale="log", ylabel="Alignnment\nerror")

ax = axes[1]
ax.plot(tser_example[tslice]*dt_u/1000/60, np.mean(learnt_pca_values[tslice], axis=1), label="Diagonal", color=back_palette[0])
ax.plot(tser_example[tslice]*dt_u/1000/60, off_diag_l_pca[tslice], label="Off-diagonal", color=back_palette[1])
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 1.1), frameon=False, handlelength=1.0, ncols=2, columnspacing=1.0)
ax.set(ylabel=r"$L_{ij}$ magnitude", xlabel="Time (min)", yscale="log")
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_robustness_biopca_align_error.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel E-F: IBCM and BioPCA neuron responses to orthogonal and parallel background components

This needs some explanation: the response to the orthogonal component goes to zero immediately, because of lateral inhibitory coupling between neurons? All neurons cancel each other's response to that background part. 

In [ ]:
def moving_average(points, kernelsize, boundary="free"):
    r""" Moving average filtering on the array of experimental points,
    averages over a block of size kernelsize.
    kernelsize should be an odd number; otherwise,
    the odd number just lower is used.
    The ith smoothed value, S_i, is:
        $$ S_i = \frac{1}{kernelsize} \sum_{j = i-kernelsize//2}^{i + kernelsize//2} x_j $$
    Values at the boundary are smoothed with smaller and smaller kernels
    (up to size 1 for boundary values)

    Args:
        points (1darray): the experimental data points
        kernelsize (int): odd integer giving the total number of points summed
        boundary (str): how to deal with points within kernelsize//2 of edges
            "shrink": the window for a point within distance d < w
                is shrunk symmetrically to a kernel of size d
            "free": the window is asymmetric, full on the inside and clipped
                on the side near the edge.
            "noflux": these points are set to the value of the closest point
                with full window (i.e. distance kernelsize//2 of the edge)

    Returns:
        smoothed (ndarray): the smoothed data points.
    """
    smoothed = np.zeros(points.shape)
    if kernelsize % 2 == 0:  # if an even number was given
        kernelsize -= 1
    w = kernelsize // 2  # width
    end = smoothed.shape[0]  # index of the last element

    if boundary not in ["shrink", "free", "noflux"]:
        raise ValueError("Unknown boundary {}".format(boundary))

    # Smooth the middle points using slicing.
    smoothed[w:end - w] = points[w:end - w]
    for j in range(w):  # Add points around the middle one
        smoothed[w:-w] += points[w - j - 1:end - w - j - 1] + points[w + j + 1:end - w + j + 1]

        # Use the loop to treat the two points at a distance j from boundaries
        if j < w and boundary == "shrink":
            smoothed[j] = np.sum(points[0:2*j + 1], axis=0) / (2*j + 1)
            smoothed[-j - 1] = np.sum(points[-2*j - 1:], axis=0)/(2*j + 1)
        elif j < w and boundary == "free":
            smoothed[j] = np.sum(points[0:j + w + 1], axis=0) / (j + w + 1)
            smoothed[-j - 1] = np.sum(points[-j - w - 1:], axis=0) / (j + w + 1)

    # Normalize the middle points
    smoothed[w:end - w] = smoothed[w:end - w] / kernelsize

    # If noflux boundary, set edge points
    if boundary == "noflux":
        smoothed[:w] = smoothed[w]
        smoothed[-w:] = smoothed[-w - 1]

    return smoothed

In [ ]:
# IBCM:
hser_orthog_ibcm = ex["hser_orthog_back"]
hser_parallel_ibcm = ex["hser_parallel_back"]
orthog_noise_norm_ibcm = ex["orthog_noise_norm"]
parallel_noise_norm_ibcm = ex["parallel_noise_norm"]
save_skp = ex["save_skp"]

# BioPCA:
hser_orthog_biopca = ex_pca["hser_orthog_back"]
hser_parallel_biopca = ex_pca["hser_parallel_back"]
orthog_noise_norm_biopca = ex_pca["orthog_noise_norm"]
parallel_noise_norm_biopca = ex_pca["parallel_noise_norm"]

In [ ]:
# Smooth the average response of each neuron in each model to orthogonal and parallel components
# IBCM first, on one plot
fig, axes = plt.subplots(2, 1, sharex=True, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.05, plt.rcParams["figure.figsize"][1])

smooth_size = 301
plot_skp = 4
tslice = slice(0, t_axis.shape[0], plot_skp)
tslice2 = slice(0, hser_parallel_ibcm.shape[0], plot_skp*save_skp)
mean_parallel_noise_norm_ibcm = np.mean(parallel_noise_norm_ibcm)
mean_orthog_noise_norm_ibcm = np.mean(orthog_noise_norm_ibcm)
neuronskip = 8
for i in range(n_i_ibcm):
    lbl = str(i) if (i % neuronskip == 0 or i == n_i_ibcm-1) else None
    # Absolute relative response (normalized by background component amplitude)
    parallel_ser = np.abs(hser_parallel_ibcm[:, i] / mean_parallel_noise_norm_ibcm)
    parallel_ser_smooth = moving_average(parallel_ser, kernelsize=smooth_size)
    axes.flat[0].plot(t_axis[tslice], parallel_ser_smooth[tslice2], color=neuron_colors_full[i], lw=0.4, alpha=0.8)
    orthog_ser = np.abs(hser_orthog_ibcm[:, i] / mean_orthog_noise_norm_ibcm)
    orthog_ser_smooth = moving_average(orthog_ser, kernelsize=smooth_size)
    axes.flat[1].plot(t_axis[tslice], orthog_ser_smooth[tslice2], color=neuron_colors_full[i], 
                      lw=0.4, alpha=0.8, label=lbl)

#axes.flat[0].plot(t_axis[tslice], moving_average(parallel_noise_norm_ibcm[tslice2], smooth_size), color="grey")
#axes.flat[1].plot(t_axis[tslice], moving_average(orthog_noise_norm_ibcm[tslice2], smooth_size), color="grey")

# Label, etc.
axes.flat[0].set_title("To parallel back. component", y=0.9)
axes.flat[1].set_title("To orthogonal back. component", y=0.9)
axes.flat[1].set_xlabel("Time (min)")
handles, labels = axes.flat[1].get_legend_handles_labels()
for h in handles:
    h.set_linewidth(1.0)
axes.flat[1].legend(handles, labels, frameon=False, fontsize=5, title_fontsize=6, title="LN index", 
                    ncols=4, columnspacing=1.0, loc="upper center")

# Trick for one label covering both axes
# add a big axis, hide frame
axlabel = fig.add_subplot(111, frameon=False)
#axlabel.set_ylabel("Smoothed, normalized\nLN activity " + r"$|\bar{h}_i| / \langle \|\mathbf{s}_{\mathrm{comp}} \| \rangle$")
axlabel.set_ylabel("Normalized LN response")
# hide tick and tick label of the big axis
axlabel.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)


avg_orthog_resp = np.mean(np.abs(hser_orthog_ibcm / mean_orthog_noise_norm_ibcm))
avg_orthog_resp_str = "{:.1e}".format(avg_orthog_resp)
resp_coef, resp_expon = avg_orthog_resp_str.split("e")
resp_expon = resp_expon[0] + resp_expon[1:].lstrip("0") 
axes.flat[1].annotate(r"$\sim" + resp_coef + r" \times 10^{" + resp_expon + "}$", 
                      xy=(t_axis[-1], avg_orthog_resp+0.1), size=6, va="bottom", ha="right")

fig.tight_layout(h_pad=1.3)
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_robustness_ibcm_response_back_components.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Then BioPCA
# Smooth the average response of each neuron in each model to orthogonal and parallel components
fig, axes = plt.subplots(2, 1, sharex=True, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.05, plt.rcParams["figure.figsize"][1])

smooth_size = 101
plot_skp = 4
tslice = slice(0, t_axis.shape[0], plot_skp)
tslice2 = slice(0, hser_parallel_biopca.shape[0], plot_skp*save_skp)
t_axis_pca = tser_example * dt_u / 1000.0 / 60.0  # min
n_i_biopca = hser_parallel_biopca.shape[1]

mean_parallel_noise_norm_biopca = np.mean(parallel_noise_norm_biopca)
mean_orthog_noise_norm_biopca = np.mean(orthog_noise_norm_biopca)
neuronskip = 4
for i in range(n_i_biopca):
    lbl = str(i) if (i % neuronskip == 0 or i == n_i_biopca-1) else None
    # Absolute relative response (normalized by background component amplitude)
    parallel_ser = np.abs(hser_parallel_biopca[:, i] / mean_parallel_noise_norm_biopca)
    parallel_ser_smooth = moving_average(parallel_ser, kernelsize=smooth_size)
    axes.flat[0].plot(t_axis_pca[tslice], parallel_ser_smooth[tslice2], color=pca_palette[i], lw=0.5, alpha=0.8)
    orthog_ser = np.abs(hser_orthog_biopca[:, i] / mean_orthog_noise_norm_biopca)
    orthog_ser_smooth = moving_average(orthog_ser, kernelsize=smooth_size)
    axes.flat[1].plot(t_axis_pca[tslice], orthog_ser_smooth[tslice2], color=pca_palette[i], 
                      lw=0.5, alpha=0.8, label=lbl)

#axes.flat[0].plot(t_axis_pca[tslice], moving_average(parallel_noise_norm_biopca[tslice2], smooth_size), color="grey")
#axes.flat[1].plot(t_axis_pca[tslice], moving_average(orthog_noise_norm_biopca[tslice2], smooth_size), color="grey")

# Label, etc.
axes.flat[0].set_title("To parallel back. component", y=0.9)
axes.flat[1].set_title("To orthogonal back. component", y=0.9)
axes.flat[1].set_xlabel("Time (min)")
handles, labels = axes.flat[1].get_legend_handles_labels()
for h in handles:
    h.set_linewidth(1.0)
axes.flat[1].legend(handles, labels, frameon=False, fontsize=5, title_fontsize=6, title="LN index", 
                    ncols=4, columnspacing=1.0, loc="upper center")

# Trick for one label covering both axes
# add a big axis, hide frame
axlabel = fig.add_subplot(111, frameon=False)
#axlabel.set_ylabel("Smoothed, normalized\nLN activity " + r"$|\bar{h}_i| / \langle \|\mathbf{s}_{\mathrm{comp}} \| \rangle$")
axlabel.set_ylabel("Normalized LN response")
# hide tick and tick label of the big axis
axlabel.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)

avg_orthog_resp = np.mean(np.abs(hser_orthog_biopca / mean_orthog_noise_norm_biopca))
avg_orthog_resp_str = "{:.1e}".format(avg_orthog_resp)
resp_coef, resp_expon = avg_orthog_resp_str.split("e")
resp_expon = resp_expon[0] + resp_expon[1:].lstrip("0") 
axes.flat[1].annotate(r"$\sim" + resp_coef + r" \times 10^{" + resp_expon + "}$", 
                      xy=(t_axis[-1], avg_orthog_resp+0.75), size=6, va="bottom", ha="right")

fig.tight_layout(h_pad=1.3)

if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_robustness_biopca_response_back_components.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel G: background inhibition of different models

In [ ]:
conc_ser_skp = conc_ser[::int(ex["skp"])]
bkvecser_example = ex_ysers["none"]
yser_optimal = ex_ysers["optimal"]  # These are already the norms
yser_avgsub = ex_ysers["avgsub"]
yser_norms = {
    "ibcm": yser_ibcm_example, 
    "avgsub": yser_avgsub,
    "biopca": yser_pca, 
    "none": bkvecser_example, 
    "optimal": yser_optimal
}
model_order = ["none", "avgsub", "biopca", "ibcm", "optimal"]
skp_plot = 1
tslice = slice(1, tser_example.shape[0], skp_plot)

fig, ax = plt.subplots()
for m in model_order:
    ax.plot(t_axis[tslice], yser_norms[m][tslice], color=model_colors[m], label=model_nice_names[m], lw=0.5)
ax.set(xlabel="Time (min)", ylabel=r"PN activity norm, $\|\mathbf{y}\|$")
ax.legend(bbox_to_anchor=(0.5, 1.0), loc="upper center", frameon=False, title="Model", ncols=2, title_fontsize=7)
ax.axhline(0.0, ls="-", color="k", lw=0.75)
ax.axhline(yser_norms["ibcm"].min(), ls="-", lw=0.5, color="k", xmin=0.95)
ax.annotate("Gap", xy=(t_axis[-1]+1.5, yser_norms["ibcm"].min()+0.1),
            xytext=(t_axis[-1]+1.5, 1.0), 
            ha="center", va="bottom", rotation=-90,
           arrowprops={"color":"k", "headwidth":2.5, "headlength":2.0, "width":0.3})
ylim = ax.get_ylim()
ax.set_ylim(ylim[0], ylim[1]+2.2)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_robustness_y_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel H: Jaccard with new odor as a function of noise amplitude

In [ ]:
jacs_file_name = pj(data_folder, "noise", "jaccard_similarities_stats_gaussnoise_identity.h5")
all_jacs_stats = pd.read_hdf(jacs_file_name, key="df")
new_concs = np.sort(all_jacs_stats.index.get_level_values("new_conc").unique())
n_new_concs = len(new_concs)
noise_ser = pd.read_hdf(jacs_file_name, key="noise_range")
all_jacs_stats

In [ ]:
back_vecs_example.shape

In [ ]:
# Average OSN activity during whiffs without noise, to compare to the noise
back_ser_nonoise = turbulent_sample["nuser"][:, :, 1].dot(back_vecs_example)
back_ser_nonoise[back_ser_nonoise == 0.0] = np.nan
median_osn_activ = np.nanmedian(back_ser_nonoise)
print(median_osn_activ)

In [ ]:
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*2.25, plt.rcParams["figure.figsize"][1])
axes = axes.flatten()
models = ["none", "avgsub", "orthogonal", "biopca", "ibcm", "optimal"]
for m in models:  # Plot IBCM last
    for i in range(n_new_concs):
        mean_jac_line = all_jacs_stats.loc[(m, slice(None), new_concs[i]), "mean"]
        std_jac_line = np.sqrt(all_jacs_stats.loc[(m, slice(None), new_concs[i]), "var"])
        axes[i].fill_between(noise_ser, 
            np.clip(mean_jac_line - std_jac_line, a_min=0, a_max=1.0), mean_jac_line + std_jac_line, 
            color=model_colors.get(m), alpha=0.4
        )
        axes[i].plot(noise_ser, mean_jac_line,
            label=model_nice_names.get(m, m), ms=4, 
            color=model_colors.get(m), alpha=1.0, marker="o"
        )
# Labeling the graphs
moments_conc = ex["moments_conc"]
print(moments_conc)
for i in range(n_new_concs):
    #axes[i].axvline(median_osn_activ, ls=":", lw=1.0, color="grey")
    axes[i].set_title("New conc. = {:.1f}".format(new_concs[i] / new_concs[1]) + r"$\langle c \rangle_\mathrm{whiff}$")
    axes[i].set_xlabel(r"Noise amplitude, $\sigma_\mathrm{OSN}$")
    axes[i].set_xscale("log")
    xlims = axes[i].get_xlim()
    axes[i].set_xlim(xlims[0]*0.3, xlims[1])
    axes[i].set_ylabel("Mean Jaccard similarity")

leg = axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False, title="Habituation", title_fontsize=6)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_robustness_jaccards_gaussnoise_amplitude.pdf"),
            transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()